## Import dependancies and set environment determinism

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''  # Hide GPU devices

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
#     tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(
    0
)
import numpy as np
import random
import pandas as pd


SEED = 378
# SEED = 123
print(SEED)
def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

# Call the above function with seed value
set_global_determinism(seed=SEED)

2024-04-24 10:10:07.666142: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


378


In [2]:
import glacierml as gl
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tqdm import tqdm

In [3]:
def run_model(model_path, n,l1,l2,loss):
            
    normalizer = preprocessing.Normalization(axis=-1)
    normalizer.adapt(np.array(trfeat[n]))

    model = gl.build_dnn_model(
        normalizer, learning_rate = 0.01, 
        layer_1 = l1, layer_2 = l2,loss = loss
    )
    model_history = model.fit(
        trfeat[n],
        trlabs[n],
        validation_split=0.2,
        callbacks = [callback],
        verbose=0, 
        epochs=500
    )
    model_filename = os.path.join(model_path)
    model.save(model_filename)
    return model

In [4]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.001,
    patience = 10,
    verbose = 0,
    mode = 'auto',
    baseline = None,
    restore_best_weights = True
)

In [5]:
home_path = '/home/simonhans/glacierml'
[
        data_path, RGI_path, glathida_path, 
        coregistration_testing_path, 
        arch_test_path, LOO_path
] = gl.set_paths(home_path)

In [6]:
# list(gl.coregister_data(data_path,'4'))

In [8]:
# df = gl.coregister_data(data_path,'4')
# df = df[[
#     'RGIId','CenLon','CenLat','Area','Zmin','Zmed','Zmax',
#     'Slope','Aspect','Lmax','Thickness'
# ]]
# df = df.sample(frac = 1,random_state = 0)
# df = df.reset_index().drop('index', axis = 1)
df = gl.coregister_data_reform(data_path,'4')
# df = gl.coregister_data(data_path,str(i))
df = df.drop(df[df['RGIId'].duplicated(keep = False)].index)
df = df.sample(frac = 1,random_state = 0)
df = df.reset_index().drop('index', axis = 1)
# df = df[list(df)[:-7]]
dft = df.copy()

trfeat = {}
trlabs = {}
tefeat = {}
telabs = {}

for n in df.index:
    mask = df.index.isin([n])
    trfeat[n] = df.loc[~mask].drop(['RGIId','Thickness'], axis = 1)
    tefeat[n] = df.loc[mask].drop(['RGIId','Thickness'], axis = 1)
    trlabs[n] = df['Thickness'].loc[~mask]
    telabs[n] = df['Thickness'].loc[mask]

model = {}
model_history = {}
normalizer = {}

for n in tqdm(df.index):
    model_path = os.path.join(LOO_path, str(n))
    if os.path.isdir(model_path) == True:
#         continue
        model[n] = gl.load_dnn_model(model_path)
    elif os.path.isdir(model_path) == False:
        set_global_determinism(seed=SEED)
        model[n] = run_model(model_path, n,l1 = 6,l2 = 2,loss = 'mae')
rgi_est_pth = os.path.join(LOO_path, 'rgi_est_raw.pkl')

RGI = gl.load_RGI_reform(RGI_path)
rfp = RGI[list(df)[:-1]]

for n in tqdm(model.keys()):
    preds = pd.Series(
        model[n].predict(rfp.drop('RGIId',axis = 1)).flatten(), name = n
    )
    RGI = pd.concat([RGI,preds], axis = 1)
RGI.to_pickle(rgi_est_pth)



  0%|                                                   | 0/273 [00:00<?, ?it/s]

6766/6766 [==============================] - 5s 691us/step


  0%|▏                                          | 1/273 [00:06<31:27,  6.94s/it]

6766/6766 [==============================] - 5s 671us/step


  1%|▎                                          | 2/273 [00:13<29:11,  6.46s/it]

...

 99%|████████████████████████████████████████▋| 271/273 [33:23<00:13,  6.64s/it]

6766/6766 [==============================] - 5s 710us/step


100%|████████████████████████████████████████▊| 272/273 [33:29<00:06,  6.58s/it]

6766/6766 [==============================] - 5s 755us/step


100%|█████████████████████████████████████████| 273/273 [33:36<00:00,  7.39s/it]


In [ ]:
df

In [ ]:
model_path

In [ ]:
LOO_path